In [1]:
import pandas as pd
import bs4 as bs
import re
import requests
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
from time import sleep

## Original Search Results

In [47]:
# import list of search strings
search_list = pd.read_csv('./sample_list.csv', skiprows=2)

In [93]:
# change spaces to %20 as prescribed in the new search url and store on list
query_list = search_list['Query'].str.replace(' ', '%20')
query_list = list(set(query_list))

In [94]:
# define query url
query_url = 'https://forum.cirruspilots.org/search?q='

In [138]:
# define site login function
def site_login():
    '''
    This is a helper function that logins to cirruspilots.org
    '''
    driver.get('https://passport.cirruspilots.org/')
    driver.find_element_by_id(
        'dnn_ctr_Login_Login_DNN_txtUsername').send_keys('jloretizo')
    driver.find_element_by_id(
        'dnn_ctr_Login_Login_DNN_txtPassword').send_keys('cth4h#@EoY*a')
    driver.find_element_by_id('login_btn').click()

In [325]:
# define the driver and execute login 
driver = webdriver.Safari()
site_login()

In [326]:
driver.get('https://www.cirruspilots.org/Forums')

In [337]:
# get the results set
results_set = {}
for i in tqdm(query_list):
    driver.get(query_url + i, )
    sleep(2)
    soup = bs.BeautifulSoup(driver.page_source, 'html.parser')
    links_list = []
    for j in soup.find_all(class_='fps-result'):
        href = j.find(class_='search-link')['href']
        links_list.append(href)
    results_set[i] = links_list
    sleep(2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def get_post_ids(row):
    ref_nums = []
    for i in range(len(row)):
        try:
            num = re.findall(r'\/[0-9]+\/[0-9]|\/[\d]+$', str(row[i]))[0]
        except:
            num = None
        ref_nums.append(num)
    return ref_nums

In [370]:
results_df = pd.DataFrame.from_dict(results_set, orient='index')
# results_df = results_df.reset_index()
results_df['ref_nums'] = results_df.apply(
    lambda row: get_post_ids(row), axis=1)

In [412]:
full_links = results_df.drop('ref_nums', axis=1).stack().reset_index().rename(
    columns={'level_0': 'search string', 0: 'id'})

In [413]:
full_links['search string'] = full_links['search string'].apply(
    lambda x: x.replace('%20', ' '))
full_links.to_csv('./results_df.csv')

In [371]:
# converting links to dictionary
links_dict = {}
for i in results_df.index:
    links_dict[i] = results_df.loc[i]['ref_nums']

In [388]:
# stacking and resetting new dataframe
link_ids = pd.DataFrame.from_dict(
    links_dict, orient='index').stack().reset_index()

In [389]:
# column rename and search string renames
link_ids['level_0'] = link_ids['level_0'].apply(
    lambda x: x.replace('%20', ' '))
link_ids = link_ids.rename(columns={'level_0': 'search string', 0: 'id'})

In [393]:
# saving data to csv
link_ids[['search string', 'id']].to_csv('./search_results.csv', index=False)

## New Search Results

In [26]:
# open the search html template
with open('./Brakes During Taxi/output4.html', 'r') as open_file:
    soup = bs.BeautifulSoup(open_file, 'html.parser')

In [4]:
search_results = pd.read_csv('./SearchApiResults.csv')

In [5]:
bdt_res = search_results.loc[search_results['Query'] == 'brakes during taxi']

In [6]:
bdt_res.head()

,Unnamed: 0,Query,rank,url,fancy_title,ContentId
0,0,brakes during taxi,1,https://forum.cirruspilots.org/t/41040/5,Cirrus SR22 Brake problems,6D601E0C-14D7-41DB-BAEF-3A6333537AB8
1,1,brakes during taxi,2,https://forum.cirruspilots.org/t/65942/2,cirrus sr22 brake wear,ADCD2721-0351-4DD7-8B07-AE423D3420F0
2,2,brakes during taxi,3,https://forum.cirruspilots.org/t/25175/7,"Overheated, Failed Brakes",2A423C4C-B3E0-4C9C-95D1-A0A86ACE1465
3,3,brakes during taxi,4,https://forum.cirruspilots.org/t/68466/1,Brake fire,AB750CAD-7103-48DC-BB4A-8B82B792B33E
4,4,brakes during taxi,5,https://forum.cirruspilots.org/t/26578/1,Brake Failure and Fire,97F47B9B-F055-4F1D-9C56-8B5FF2A3D32C


In [7]:
all_items = soup.find_all(class_='fps-result')

In [8]:
ratio1 = pd.read_csv('./ratio_comb3.csv')

In [9]:
ratio1.head()

,Unnamed: 0,Query,url,ContentId,fancy_title
0,0,brakes during taxi,https://forum.cirruspilots.org/t/19650/3,AD2ED868-83CB-4404-86EF-0072EB200FD6,frozen brakes
1,1,brakes during taxi,https://forum.cirruspilots.org/t/62895/3,AC9F1522-CA75-4BE9-AEB7-E07A8406BF34,A Place to Vent
2,2,brakes during taxi,https://forum.cirruspilots.org/t/98008/90,B6E67966-7CE4-465F-80F8-2E817735A523,"CAPS Save #91 in SR22 N578SP near Fort Wayne, ..."
3,3,brakes during taxi,https://forum.cirruspilots.org/t/16274/2,1558F623-8F1C-44FA-9596-52A741E81988,taxiing an sr20
4,4,brakes during taxi,https://forum.cirruspilots.org/t/78835/35,7D7FCE16-918F-4E94-9677-ADB24A94E297,Eclipse runway excursion


In [10]:
def get_id(item):
    res_url = re.findall(r'\/[0-9]+\/[0-9]+|\/[\d]+$',item)[0]
    return res_url

In [11]:
ratio1['url-id'] = ratio1['url'].apply(lambda x: get_id(x))

In [12]:
brakes_during_taxi = ratio1.loc[ratio1['Query'] == 'brakes during taxi']

In [13]:
ratio1.loc[ratio1['Query'] == 'brakes during taxi'].shape

(137, 6)

In [14]:
all_items = soup.find_all(class_='fps-result')
html_links = pd.DataFrame(all_items)

In [15]:
def get_href(html):
    href = html.find(class_='search-link')['href']
    return get_id(href)

In [16]:
html_links['id'] = html_links[3].apply(lambda x: get_href(x))

In [17]:
brakes_during_taxi = brakes_during_taxi.rename(columns={'Unnamed: 0': 'New Rank'})

In [18]:
brakes_during_taxi.head()

,New Rank,Query,url,ContentId,fancy_title,url-id
0,0,brakes during taxi,https://forum.cirruspilots.org/t/19650/3,AD2ED868-83CB-4404-86EF-0072EB200FD6,frozen brakes,/19650/3
1,1,brakes during taxi,https://forum.cirruspilots.org/t/62895/3,AC9F1522-CA75-4BE9-AEB7-E07A8406BF34,A Place to Vent,/62895/3
2,2,brakes during taxi,https://forum.cirruspilots.org/t/98008/90,B6E67966-7CE4-465F-80F8-2E817735A523,"CAPS Save #91 in SR22 N578SP near Fort Wayne, ...",/98008/90
3,3,brakes during taxi,https://forum.cirruspilots.org/t/16274/2,1558F623-8F1C-44FA-9596-52A741E81988,taxiing an sr20,/16274/2
4,4,brakes during taxi,https://forum.cirruspilots.org/t/78835/35,7D7FCE16-918F-4E94-9677-ADB24A94E297,Eclipse runway excursion,/78835/35


In [19]:
html_links = html_links.reset_index()
html_links = html_links.rename(columns={'index':'old_rank'})

In [20]:
brakes_during_taxi.loc[brakes_during_taxi['url-id'].str.contains('/65942/2'),'New Rank']

55    55
Name: New Rank, dtype: int64

In [21]:
html_links.head()

,old_rank,0,1,2,3,4,id
0,0,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/41040/5
1,1,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/65942/2
2,2,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/25175/7
3,3,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/68466
4,4,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/26578


In [22]:
for i in html_links['id']:
    try: 
        new_rank = brakes_during_taxi.loc[brakes_during_taxi['url-id'].str.contains(i)].index[0]
    except:
        new_rank = None
#     print(new_rank)
    html_links.loc[html_links['id'] == i, 'New Rank'] = new_rank

In [23]:
new_order = list(html_links.sort_values('New Rank', ascending=True)['old_rank'])

In [24]:
final_order = [all_items[i] for i in new_order]

In [27]:
# open the search html template
with open('./Brakes During Taxi/output4.html', 'r') as open_file:
    soup = bs.BeautifulSoup(open_file, 'html.parser')
for i in tqdm(range(150)):
#     print(i)
    soup.find_all(class_='fps-result')[i].replace_with(final_order[i])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [28]:
with open("./Brakes During Taxi/output3.html", "w") as file:
    file.write(str(soup))

In [166]:
for i in soup.find(id='ember131').children:
    print(i)

 
<div class="fps-result">
<div class="author">
<a data-user-card="steveg" href="https://forum.cirruspilots.org/u/steveg">
<img alt="" aria-label="Steven Grantz" class="avatar" height="45" src="./Search results for 'brakes during taxi' - Cirrus Owners and Pilots Association - https___forum.cirruspilots.org__files/84_2.png" title="Steven Grantz" width="45"/>
</a>
</div>
<div class="fps-topic">
<div class="topic">
<!-- -->
<a class="search-link" data-ember-action="" data-ember-action-133="133" href="https://forum.cirruspilots.org/t/cirrus-sr22-brake-problems/41040/5">
<div class="topic-statuses ember-view" id="ember134"><!-- --><!-- --><!-- --><!-- --><!-- --><!-- --><!-- --><!-- --></div>
<span class="topic-title"><span class="ember-view" id="ember135">Cirrus SR22 Brake problems</span></span>
</a>
<div class="search-category">
<!-- --> <a class="badge-wrapper bullet" href="https://forum.cirruspilots.org/c/guest-discussion/29"><span class="badge-category-bg" style="background-color: #231

</div>


<div class="fps-result">
<div class="author">
<a data-user-card="jtberlin" href="https://forum.cirruspilots.org/u/jtberlin">
<img alt="" aria-label="Jeffrey Berlin" class="avatar" height="45" src="./Search results for 'brakes during taxi' - Cirrus Owners and Pilots Association - https___forum.cirruspilots.org__files/45(23).png" title="Jeffrey Berlin" width="45"/>
</a>
</div>
<div class="fps-topic">
<div class="topic">
<!-- -->
<a class="search-link" data-ember-action="" data-ember-action-816="816" href="https://forum.cirruspilots.org/t/food-for-thought-from-a-sage-master/29129">
<div class="topic-statuses ember-view" id="ember817"><!-- --><!-- --><!-- --><!-- --><!-- --><!-- --><!-- --><!-- --></div>
<span class="topic-title"><span class="ember-view" id="ember818">Food for thought from a sage master</span></span>
</a>
<div class="search-category">
<!-- --> <a class="badge-wrapper bullet" href="https://forum.cirruspilots.org/c/cirrus-flying/7"><span class="badge-category-bg" st

In [164]:
new_html = ''
for i in [all_items[i] for i in new_order]:
    new_html += i

TypeError: can only concatenate str (not "Tag") to str

In [146]:
brakes_during_taxi.loc[brakes_during_taxi['url-id'].str.contains('/41040/5')].index[0]

40

In [98]:
brakes_during_taxi.loc[brakes_during_taxi['url-id'].str.contains('/41040/5')]['New Rank']

40    40
Name: New Rank, dtype: int64

In [84]:
html_links.head()

,old_rank,0,1,2,3,4,id
0,0,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/41040/5
1,1,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/65942/2
2,2,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/25175/7
3,3,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/68466
4,4,\n,"[\n, [\n, [], \n], \n]",\n,"[\n, [\n, , \n, [\n, <div class=""topic-status...",\n,/26578


In [66]:
pd.DataFrame(all_items)[3][3].find(class_='search-link')['href']

'https://forum.cirruspilots.org/t/brake-fire/68466'

In [23]:
 soup.find_all(class_='fps-result')[0].find(class_='search-link')['href']

True

In [37]:
# find all content item classes
for i in soup.find_all(class_='content-item'):
    # change the value of title inside the a tag
    i.find(class_='abbreviated-post').find('h4').find('a')['title'] = 'New Title'
    # change the href for link
    i.find(class_='abbreviated-post').find('h4').find('a')['href'] = 'New Title'
    # change text value
    i.find(class_='abbreviated-post').find('h4').find('a').string.replace_with("No longer bold")
    

In [38]:
# save this new html as test panel
with open("output1.html", "w") as file:
    file.write(str(soup))

In [41]:
soup.find_all(class_='content-item')[0].find('h4')

<h4 class="post-name"><a class="internal-link view-post" href="https://www.cirruspilots.org/copa/safety_programs/f/349/t/159914.aspx" title="New Title">No longer bold</a></h4>